In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
df = pd.read_csv('rgi_11.csv')
df

,Unnamed: 0,rgi_id,w5e5_hgt,w5e5_coords,month,terminus_alt,area,temp,gradient
0,0,RGI60-11.00001,930.0,"[47.75, 13.75]",1,2003.0,0.122,-8.205430,-0.005373
1,1,RGI60-11.00001,930.0,"[47.75, 13.75]",2,2003.0,0.122,-7.247490,-0.005580
2,2,RGI60-11.00001,930.0,"[47.75, 13.75]",3,2003.0,0.122,-4.579250,-0.005796
3,3,RGI60-11.00001,930.0,"[47.75, 13.75]",4,2003.0,0.122,-0.561740,-0.006106
4,4,RGI60-11.00001,930.0,"[47.75, 13.75]",5,2003.0,0.122,4.362788,-0.006195
...,...,...,...,...,...,...,...,...,...
38803,38803,RGI60-11.03234,1660.0,"[42.75, -0.25]",8,2255.0,0.025,12.704171,-0.006094
38804,38804,RGI60-11.03234,1660.0,"[42.75, -0.25]",9,2255.0,0.025,9.403811,-0.005876
38805,38805,RGI60-11.03234,1660.0,"[42.75, -0.25]",10,2255.0,0.025,5.235299,-0.005756
38806,38806,RGI60-11.03234,1660.0,"[42.75, -0.25]",11,2255.0,0.025,-0.569548,-0.005596


In [2]:
df2 = df[df.month==1]
df2 = df2.reset_index()
for i in range(0,df2.shape[0]):
    df2.loc[i, "subzero_alt"] = -df2.loc[i,"temp"]/df2.loc[i,"gradient"] + df2.loc[i,"terminus_alt"]
df2[["rgi_id", "terminus_alt", "subzero_alt", "temp", "gradient"]]

,rgi_id,terminus_alt,subzero_alt,temp,gradient
0,RGI60-11.00001,2003.0,475.892696,-8.205430,-0.005373
1,RGI60-11.00002,1253.0,475.892696,-4.175541,-0.005373
2,RGI60-11.00003,1478.0,475.892696,-5.384508,-0.005373
3,RGI60-11.00004,1825.0,475.892696,-7.249003,-0.005373
4,RGI60-11.00005,1721.0,475.892696,-6.690192,-0.005373
...,...,...,...,...,...
3229,RGI60-11.03230,2205.0,1496.120251,-3.979579,-0.005614
3230,RGI60-11.03231,2139.0,1496.120251,-3.609062,-0.005614
3231,RGI60-11.03232,1873.0,1496.120251,-2.115765,-0.005614
3232,RGI60-11.03233,2309.0,1496.120251,-4.563424,-0.005614


In [3]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=df.loc[x.index, "area"])
# Groupby and aggregate with namedAgg [1]:
df3 = df2.groupby("w5e5_hgt").agg(weighted_terminus_alt=("terminus_alt", wm), weighted_subzero_alt=("subzero_alt", wm), weighted_temp=("temp", wm))
df3 = df3.reset_index()
for i in range(0, df3.shape[0]):
    df3.loc[i, "w5e5_coords"] = df2.loc[df2.w5e5_hgt == df3.w5e5_hgt[i], "w5e5_coords"].values[0]
df3

,w5e5_hgt,weighted_terminus_alt,weighted_subzero_alt,weighted_temp,w5e5_coords
0,741.0,823.000000,759.321360,-0.347586,"[47.25, 8.75]"
1,930.0,1618.285714,475.864713,-6.138142,"[47.75, 13.75]"
2,984.0,974.244075,870.415954,-0.566740,"[47.25, 9.25]"
3,1070.0,1716.346955,707.373884,-5.564690,"[47.25, 9.75]"
4,1177.0,1995.046098,859.483528,-6.330099,"[46.75, 7.75]"
5,1179.0,1687.639720,1038.190325,-3.597975,"[46.25, 6.75]"
6,1419.0,2098.670281,732.942629,-7.445722,"[46.25, 9.25]"
7,1513.0,1462.000000,1710.435836,1.396731,"[42.75, 1.25]"
8,1598.0,2020.875785,819.111795,-6.513846,"[47.25, 13.25]"
9,1628.0,2090.377401,887.899648,-6.592650,"[47.25, 11.75]"


In [4]:
df4 = df3[['w5e5_hgt', 'w5e5_coords', 'weighted_subzero_alt', 'weighted_terminus_alt', 'weighted_temp']]
df4['w5e5_coords'] = df4['w5e5_coords'].astype(str)
df4.dtypes
df5 = df4.groupby("w5e5_hgt").mean().reset_index()
for i in range(0,df5.shape[0]):
    df5.loc[i,"w5e5_coords"] = df4.loc[df4.w5e5_hgt==df5.w5e5_hgt[i], "w5e5_coords"].values[0]

df5["alt_diff"] = df5["weighted_terminus_alt"] - df5["weighted_subzero_alt"]
df6 = df5[['w5e5_coords', 'alt_diff', 'weighted_temp']]
df6['w5e5_coords']=df6['w5e5_coords'].str.strip('[]').str.split(',')
df6['lat'], df6['lon'] = zip(*df6.pop('w5e5_coords'))
for i in range(0,df6.shape[0]):
    df6.loc[i,"slope"] = ds2.sel(latitude=df6.lat[i], longitude=df6.lon[i], method="nearest").RiverSlope.values[0]
df6 = df6[["lat", "lon", "alt_diff", "slope", "weighted_temp"]]
df6

/tmp/ipykernel_11034/1611354979.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['w5e5_coords']=df6['w5e5_coords'].str.strip('[]').str.split(',')
/tmp/ipykernel_11034/1611354979.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['lat'], df6['lon'] = zip(*df6.pop('w5e5_coords'))
/tmp/ipykernel_11034/1611354979.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

NameError: name 'ds2' is not defined

In [ ]:
ds2 = xr.open_dataset("/home/suryab/work/w5e5_orog/Global_RiverSlope_HydroSTN30_06min_Static.nc", decode_times=False)

mask1=  (df6.alt_diff > 10)
mask2 = (df6.slope > 0.01)
df6[mask2 & mask1].count()

In [ ]:
plt.scatter(df6.index, df6.alt_diff, color="black", label = "Condition 1 failure")
plt.scatter(df6[mask1].index, df6[mask1].alt_diff, color="blue", label = "Condition 2 failure")
plt.scatter(df6[mask2].index, df6[mask2].alt_diff, color="red", label = "All conditions pass")
plt.ylabel("Terminus and Subzero alt difference")
plt.xlabel("W5E5 grid cells")
plt.legend()

In [ ]:
plt.scatter(df6.index, df6.slope, color="black")
plt.ylabel("Slope")
plt.xlabel("W5E5 grid cells")
plt.legend()